In [ ]:
!pip install openpyxl


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Define the path to the uploaded file
file_path = "/content/Online Retail.xlsx"

# Load the dataset into a DataFrame
df = pd.read_excel(file_path, engine='openpyxl')

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Shape of the dataset
print("Shape:", df.shape)

# Check column types and nulls
df.info()

Shape: (541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
# Summary statistics
df.describe().T



,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


In [ ]:
# Count missing values
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
# Standardize InvoiceNo format
df['InvoiceNo'] = df['InvoiceNo'].astype(str).str.upper().str.strip()

# Capture returns before any cleaning removes them
df_returns = df[df['InvoiceNo'].str.startswith('C') | (df['Quantity'] < 0)].copy()

print("Returns detected before cleaning:", df_returns.shape)

Returns detected before cleaning: (10624, 8)


In [ ]:
# Count full-row duplicates
full_duplicates = df.duplicated()
print("Total full-row duplicates:", full_duplicates.sum())


Total full-row duplicates: 5268


In [ ]:
df[full_duplicates].head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom


In [ ]:
# Remove full-row duplicates
df_cleaned = df.drop_duplicates()

# Show new shape
print("Shape after removing duplicates:", df_cleaned.shape)


Shape after removing duplicates: (536641, 8)


In [ ]:
# Drop rows with missing customer IDs
df_cleaned = df_cleaned.dropna(subset=['CustomerID'])

# Show new shape
print("Shape after removing missing customer IDs:", df_cleaned.shape)


Shape after removing missing customer IDs: (401604, 8)


In [ ]:
# Keep only rows with positive Quantity and UnitPrice
df_cleaned = df_cleaned[(df_cleaned['Quantity'] > 0) & (df_cleaned['UnitPrice'] > 0)]

# Show new shape
print("Shape after removing invalid Quantity/Price:", df_cleaned.shape)


Shape after removing invalid Quantity/Price: (392692, 8)


In [ ]:
# Reset index after all drops
df_cleaned.reset_index(drop=True, inplace=True)


In [ ]:
df_cleaned.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df_returns.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [ ]:
# Customers table
df_customers = df_cleaned[['CustomerID', 'Country']].drop_duplicates()

# Products table
df_products = df_cleaned[['StockCode', 'Description']].drop_duplicates()

# Invoices table
df_invoices = df_cleaned[['InvoiceNo', 'InvoiceDate']].drop_duplicates()

# Transactions (fact table)
df_transactions = df_cleaned[['InvoiceNo', 'StockCode', 'CustomerID', 'Quantity', 'UnitPrice']]


In [ ]:
# Export Customers table
df_customers.to_csv("customers.csv", index=False)

# Export Products table
df_products.to_csv("products.csv", index=False)

# Export Invoices table
df_invoices.to_csv("invoices.csv", index=False)

# Export Transactions table
df_transactions.to_csv("transactions.csv", index=False)


In [ ]:
# Filter out rows with missing CustomerID or StockCode
returns_partial = df_returns[
    df_returns['CustomerID'].notnull() &
    df_returns['StockCode'].notnull()
].copy()

# Clean column names
returns_partial.columns = [col.strip() for col in returns_partial.columns]
returns_partial = returns_partial.loc[:, ~returns_partial.columns.duplicated()]
returns_partial = returns_partial.rename(columns={'UnitPrice': 'Unit_Price'})

# Export clean returns file
returns_partial.to_csv("returns_import_ready.csv", index=False, encoding='utf-8')

# Confirm export
print("✅ Final return rows exported:", len(returns_partial))


✅ Final return rows exported: 8905


In [ ]:
returns_partial.to_csv("returns_import_ready.csv", index=False, encoding='utf-8')

print(returns_partial.columns.tolist())


['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Unit_Price', 'CustomerID', 'Country']
